In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time
import re

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--lang=ko_KR")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(10)
driver.maximize_window()

wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

urls = [
    # "https://www.youtube.com/watch?v=jpKAf-y3psg&list=PLWoGc25qzA3tNkDJ6_RQzsTemA4RFcScb",
    # "https://www.youtube.com/watch?v=bWN3ZIV0BPQ&list=PLWoGc25qzA3vthfzjd7LAS24_A58iIKZv",
    # "https://www.youtube.com/watch?v=0xAHCaHWbKk&list=PLWoGc25qzA3s__HwGKJphyLl_KZTEwYGS",
    # "https://www.youtube.com/watch?v=rUFJ62QhMZM&list=PLWoGc25qzA3vAvOy52DJN7vco_jkU0cV2",
    "https://www.youtube.com/watch?v=dOGpXEOiYH8&list=PLWoGc25qzA3vRoh0umDUCk3FnWjk6IKhF",
    "https://www.youtube.com/watch?v=MzPvmiSKcj8&list=PLWoGc25qzA3s5wC1AdhHf1LGojWrE8pOa",
    "https://www.youtube.com/watch?v=pOV-txFfYDE&list=PLWoGc25qzA3uNHuMdjiRyYJVVY-_Yq3TA",
    "https://www.youtube.com/watch?v=NSuwkeG_ro0&list=PLWoGc25qzA3tSRZKQVLoE7NEcUfQFaMLN",
    "https://www.youtube.com/watch?v=EFB18WyD1G0&list=PLWoGc25qzA3tAoQ3-7uAt7ZywMAMwbPPK"
]
prev_idx = 0
datas = list()
for url in urls:
    driver.get(url)
    time.sleep(0.5)

    playlists = driver.find_element(
        By.XPATH,
        "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/ytd-playlist-panel-renderer/div/div[3]",
    )
    title_elems = playlists.find_elements(By.ID, "video-title")
    for title in title_elems:
        # title = title.text.split("|")[-1]
        title = title.text
        datas.append([title])

    time.sleep(3)
    run_time = playlists.find_elements(By.CLASS_NAME, "badge-shape-wiz__text")
    print(len(run_time))
    for i, r_time in enumerate(run_time):
        i = i + prev_idx
        r_time = r_time.text
        datas[i].append(r_time)

    link_imgs = playlists.find_elements(By.CSS_SELECTOR, "img")
    print(len(link_imgs))
    for i, link in enumerate(link_imgs):
        i = i + prev_idx
        action.move_to_element(link).perform()
        link.click()

        time.sleep(1)
        try:
            premium = driver.find_element(By.XPATH,'//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
            premium.click()
        except Exception as e:
            pass
        likes = driver.find_element(
            By.XPATH,
            '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]',
        )
        likes = likes.text  # 1000 단위는 K 로 나타나 정확한 수치를 알 수 없음
        datas[i].append(likes)

        expand_button = driver.find_element(By.ID, "expand")
        expand_button.click()
        # 조회수
        views = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
        views = int(re.sub(r"[가-힣,\s]", "", views.text))
        datas[i].append(views)
        # DATETIME
        date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
        date = re.sub(r"[가-힣,.:]", "", date.text).strip().split()
        for d in date:
            datas[i].append(d)
        
        texts = list()
        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (
                        By.XPATH,
                        '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button',
                    )
                )
            )
            action.move_to_element(button).perform()
            button.click()

            wait.until(EC.presence_of_element_located((By.ID, "secondary")))
            time.sleep(3)

            # 페이지 소스 가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 자막 세그먼트 모두 가져오기
            segments = soup.find_all(
                "div", class_="segment style-scope ytd-transcript-segment-renderer"
            )

            # 자막 시간과 텍스트 추출
            for segment in segments:
                text = segment.find(
                    "yt-formatted-string",
                    class_="segment-text style-scope ytd-transcript-segment-renderer",
                ).get_text(strip=True)
                texts.append(text)

        except Exception as e:
            texts = list()
            pass
        datas[i].append(texts)
    prev_idx += len(run_time)


driver.quit()

18
18
11
11
27
27
17
17
15
15


In [2]:
print(datas[0])

['최자로드5 EP.18 | 미슐랭 빕구르망 3년 연속 선정 돼지고기', '22:25', '2.1천', 257262, '2021', '12', '23', ['뭐 할 수 말씀으로', '뭐 싼타모 좀 약간 이사람 배 한데서', '[음악]', '여기 시장도 있고 올해는 동네', '기도하고 넓게 봤을 때 여기 다', '좋은데 가 되게 많은 도움 해 되시면', '멋집 나오는 동네 여기 재미있지', '[음악]', '으', '어 여기 너무 유명한 집에서 굳이', '나까지 소개도 되나 싶을 정도로', '유명한 집인데 내가 생각할때 서울', '3대 돼지고기 집중의 하나인', '구내식당의 1차', '우리 모두가 제일 많이 먹는게 사실', '삼겹살이 잡아 이 삼겹살은 보기', '자체도 일단 너무 좋고 오래 온도도', '너무 적고 부웅 정도로 너무 좋고 다', '열을 했을 때 쫀쫀 해지면서 기름이', '그대로 있는 스타일이라고 해서 나타', '이번 시집 벌써 만나 가져가게 마지막', '했는데 오늘은 이 카드를 써야 겠다', '싶어 가지고 1 2급 1호 밀려서', '베이스가 마지막에 나온 느낌', '보임이 불렀지', '개구리 자기만의 먹음 에 처음 있거든', '처리는 대시를 좋아합니다 생각하는', '대준이 1대 1 뭐 하는 사람', '맛있는걸 그것만으로 행복한 연말을', '보내는 시기 아니까 연말에 2', '지정되어 있지 않은 것으로 계속', '보기로 또', '페이즈', '[음악]', '안전으로 에러 부족한 부분이 있어도', '마신의 오직 채워줘 예 멋진 아닐 때', '모아 왔어요 더욱이나 3 넉살이 진짜', '맛있어요 옹 상법이 돼지가 좀 다르고', '는 조금 신화의 질감이 싫게 하다고', '애가 이것을 먹어 봐야 돼 실도 좀', '먹고 왔어 그래 또 지금이야 여기어', '안녕하세요 안녕하세요 에 조문 전에', '쿄코는 체크인 먼저 봐야 겠', '네 부모는 어떻게 될까요 일단 저희', '산격 뜨게 추석에 생각들이 1에는', '아시 거리는게 자 그냥 마실 버리는', '요 또 소주 전

In [3]:
print(len(datas))

88


In [4]:
import pandas as pd

df = pd.DataFrame(
    datas,
    columns=["title", "run_time", "likes", "views", "year", "month", "day", "script"],
)
print(df.tail(5))

                                                title run_time likes   views  \
83  통닭에 마라와 가쓰오부시를 끼얹나? 장작구이 통닭의 리믹스Ver. (ft. 코카-콜...    30:18   672  676100   
84  잡는 순간 올라! 멕시코 정통 타코에 코카-콜라 페어링 (w/개코, pH-1, JU...    32:35   710  604886   
85  믿었던 돼지의 뒤통수. 이 맛을 지금 알아 억울한 김해 뒷통수육(w/맛타고라스) |...    31:42   802  311168   
86  콧등부터 꼬리까지. 몰래 빼먹던 김해뒷고기의 전국구 선언 (w/맛타고라스) | 최자...    29:11   924  378677   
87                      어센틱한 맛을 찾아 떠나는 혓바닥 여행 | 최자로드9     1:17   257   14133   

    year month day                                             script  
83  2024     6   6  [와이프도 음식을 엄청 좋아해 약간 제 와이프가 여자자 같이, 만나면 계속 먹는, ...  
84  2024     5  30  [그럼 보통 몇 타코 정도해 한번 먹을 때요 정도 맛에요 정도 크기면 한 다섯 개 ...  
85  2024     5  23  [우리의 일주일은 월 봤수 모 금금금 가득 사이 너무 때문에 실 틈 틈틈 없어졌어 ...  
86  2024     5  16  [[음악], 여행에 즐거운 점은 언제나 그렇듯이, 새로운 데를 가보고 새로운 걸, ...  
87  2024     5   9  [[박수], 시즌 나 기다리고, 계셨나요, 우 휴식기가, 필요한게 다시 카메라 켜도...  


In [9]:
# SAVE
df.to_csv('result3.csv',index=False)

In [10]:
df1 = pd.read_csv('result.csv')
df2 = pd.read_csv('result2.csv')
df3 = pd.read_csv('result3.csv')

In [16]:
df_fin = pd.concat([df1,df2,df3],ignore_index=True)
print(df_fin.tail(10))

                                                 title run_time likes  \
286     평양냉면 인플레 시대. 만원으로 즐기는 56년 전통의 맛 | 최자로드9 EP. 09    23:50  1.2천   
287  좌시경 우최자 | 한 여름 낮의 중식과 고량주... 너흰 감동이었어! w/@sung...    38:39  6.5천   
288  "오늘 준비한 고기 태우는 육식맨부터 보시죠" 폼 미친 삼겹살과 정성 넘치는 반찬 ...    31:09  1.1천   
289  [라면꼰대 X 최자로드] "드디어 만났습니다" 마파두부, 나폴리탄으로 발우공양 | ...    35:11  2.5천   
290  막창순대 업고 튀어! 돈설 수육까지 완벽한 덕질유발 순대덕후의 집(w/김재환, Pa...    33:21  1.3천   
291  통닭에 마라와 가쓰오부시를 끼얹나? 장작구이 통닭의 리믹스Ver. (ft. 코카-콜...    30:18   672   
292  잡는 순간 올라! 멕시코 정통 타코에 코카-콜라 페어링 (w/개코, pH-1, JU...    32:35   710   
293  믿었던 돼지의 뒤통수. 이 맛을 지금 알아 억울한 김해 뒷통수육(w/맛타고라스) |...    31:42   802   
294  콧등부터 꼬리까지. 몰래 빼먹던 김해뒷고기의 전국구 선언 (w/맛타고라스) | 최자...    29:11   924   
295                      어센틱한 맛을 찾아 떠나는 혓바닥 여행 | 최자로드9     1:17   257   

        views    year  month   day  \
286  311205.0  2024.0    8.0   8.0   
287  837965.0  2024.0    7.0   4.0   
288  362006.0  2024.0    6.0  27.0   
289  408005.0  2024.0    6.0  20.0   
290  2

In [18]:
df_fin.to_csv('final_.csv',index=False)

In [20]:
df1 = pd.read_csv('final_.csv')
df2 = pd.read_csv('final.csv')
df2.drop(columns=['comments_number','upload_date', 'likes_number', 'running_time', 'script', 'views_number','minmax_views'],inplace=True)
print(df2.columns)

Index(['content_name', 'chennel', 'local', 'restaurant_type', 'guest_info',
       'food_scenes', 'eating_scenes', 'drinking_scene',
       'restaurant_info_scene', 'conversation_scene'],
      dtype='object')


In [21]:
df = pd.merge(df1,df2,left_on='title',right_on='content_name')

Index(['title', 'run_time', 'likes', 'views', 'year', 'month', 'day', 'script',
       'content_name', 'chennel', 'local', 'restaurant_type', 'guest_info',
       'food_scenes', 'eating_scenes', 'drinking_scene',
       'restaurant_info_scene', 'conversation_scene'],
      dtype='object')
240


In [26]:
df.drop(columns=['content_name'],inplace=True)
print(df.head(3))
df.to_csv('data.csv',index=False)

                           title run_time likes      views    year  month  \
0       [sub] 성시경의 먹을텐데 l 부산 만우장    24:17  1.2만  1040907.0  2024.0    8.0   
1     [sub] 성시경의 먹을텐데 l 부산 해진아나고    27:33    1만   712699.0  2024.0    8.0   
2  [sub] 성시경의 먹을텐데 l 속초 화진호 이선장네    34:42  1.5만  1029494.0  2024.0    8.0   

    day                                             script  chennel   local  \
0  30.0  ["[SUNG SI KYUNG's Grab a Bite]", 'Okay, for t...      0.0  부산 수영구   
1  26.0  ["[The audio quality isn't great due to losing...      0.0  부산 수영구   
2  21.0  ["[SUNG SI KYUNG's Grab a Bite] Oh, my voice. ...      0.0  강원 속초시   

  restaurant_type guest_info  food_scenes  eating_scenes  drinking_scene  \
0              중식         없음          1.0            0.0             0.0   
1           고기·구이         없음          0.0            0.0             1.0   
2              한식         없음          1.0            0.0             0.0   

   restaurant_info_scene  conversation_scene  
0                    0